# Data Wrangling and explorative data analysis

### Imports

In [1]:
import numpy as np
import __main__ as main

from pathlib import Path
from joblib import load, dump
from helpers.is_interactive import is_interactive

import warnings
warnings.filterwarnings('ignore')

### Run dependency notebooks

In [2]:
if is_interactive(main):
    %run 01_0_data_wrangling.ipynb -p
    %run 01_2_data_wrangling_kaggle.ipynb -p
    %run 02_0_scaling.ipynb -p

Running previous notebooks...


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Index


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Index


### CSV Import

In [65]:
source_path = Path('./data/kaggle/01_clean_data.pkl')
source_data = load(source_path)

df = source_data['dataset']
df = df.set_index("Index")

In [66]:
df.tail()

,Living space,Plot area,Floor,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Longitude,NoisePollutionRailwayL,NoisePollutionRailwayM,...,type_unified_penthouse,type_unified_rustico,type_unified_secondary-suite,type_unified_semi-detached-house,type_unified_single-room,type_unified_stepped-apartment,type_unified_stepped-house,type_unified_studio,type_unified_terrace-house,type_unified_villa
0,220.0,733.0,1.0,0.164382,0.100030,0.063548,47.372110,8.075858,0.003811,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,230.0,702.0,1.0,0.260855,0.170434,0.083253,47.371558,8.073110,0.002623,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,131.0,131.0,1.0,0.434114,0.357984,0.125505,47.413754,8.082318,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,140.0,206.0,1.0,0.148190,0.076610,0.000000,47.373327,8.076892,0.005193,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,156.0,222.0,1.0,0.511176,0.286451,0.090908,47.415927,8.085840,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Scaling
Using the Min-Max-Method

In [67]:
scaled_df = df.copy()
source_path = Path('data/model/02_train_data.pkl')
scaler = load(source_path)['scaler']

columns = scaled_df.columns

scaled_df[columns] = scaler.transform(scaled_df[columns])

scaled_df.head()

,Living space,Plot area,Floor,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Longitude,NoisePollutionRailwayL,NoisePollutionRailwayM,...,type_unified_penthouse,type_unified_rustico,type_unified_secondary-suite,type_unified_semi-detached-house,type_unified_single-room,type_unified_stepped-apartment,type_unified_stepped-house,type_unified_studio,type_unified_terrace-house,type_unified_villa
0,0.026770,0.002072,0.178571,0.183264,0.100085,0.063548,0.786382,0.475555,0.014734,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.028021,0.001983,0.178571,0.290820,0.170527,0.083253,0.786102,0.474934,0.010139,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.015637,0.000356,0.178571,0.483981,0.358180,0.125505,0.807571,0.477015,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.016763,0.000570,0.178571,0.165213,0.076652,0.000000,0.787002,0.475789,0.020076,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.018764,0.000616,0.178571,0.569894,0.286608,0.090908,0.808677,0.477811,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Enhancing the dataset with additional columns with the power of an existing column

In [68]:
numeric_columns = scaled_df.select_dtypes(include=np.number).columns.tolist()

numeric_df = scaled_df[numeric_columns]

numeric_df = numeric_df.drop(columns=[
    col
    for col in numeric_columns
    if 'type_' in col
])

numeric_columns = numeric_df.columns.tolist()

In [69]:
clean_df = scaled_df.copy()

for col in numeric_columns:
    for p in range(2,7):
        clean_df[f'{col}_{p}'] = numeric_df[col]**p


### Export csv

In [70]:
target_path = Path('./data/kaggle/02_model_data.pkl')
dump({
    'dataset': clean_df,
    'imputers': source_data['imputers'],
    'base_imputer': source_data['base_imputer'],
    'scaler': scaler
}, target_path)

['data/kaggle/02_model_data.pkl']